In [60]:
!pip install ipython-autotime
%load_ext autotime
%pylab inline 

import numpy as np
import gym
import random
import time

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
Populating the interactive namespace from numpy and matplotlib
time: 2.04 s


/usr/local/lib/python3.6/dist-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['info', 'random', 'gamma', 'step']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [90]:
# Environment
env = gym.make("Taxi-v3")

# Training parameters for Q learning
alpha = 0.7                   # Learning rate
gamma = 0.6                   # Future reward discount factor

num_of_episodes = 10000        # Total episodes
num_of_steps = 500            # per each episode

# Exploration parameters
epsilon = 1                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.01            # Minimum exploration probability 
decay_rate = 0.01             # Exponential decay rate for exploration prob

# Get to know the environment
action_size = env.action_space.n
print("Action size ", action_size)

state_size = env.observation_space.n
print("State size ", state_size)


Action size  6
State size  500
time: 72.7 ms


In [91]:
# Q tables for rewards
Q_reward = np.zeros((state_size,action_size))

time: 1.13 ms


In [93]:
# Training w/ random sampling of actions
# YOU WRITE YOUR CODE HERE

for episode in range(num_of_episodes):
  # Reset the environment
  state = env.reset()
  step = 0
  done = False
    
  for step in range(num_of_steps):
    # 3. Choose an action a in the current world state (s)
    ## First we randomize a number
    exp_exp_tradeoff = random.uniform(0,1)
        
    ## If this number > greater than epsilon --> exploitation (taking the biggest Q value for this state)
    if exp_exp_tradeoff > epsilon:
      action = np.argmax(Q_reward[state,:])
        
    # Else doing a random choice --> exploration
    else:
      action = env.action_space.sample()
        
    # Take the action (a) and observe the outcome state(s') and reward (r)
    new_state, reward, done, info = env.step(action)

    # Update Q(s,a):= Q(s,a) + alpha * [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
    Q_reward[state, action] = Q_reward[state, action] + alpha * (reward + gamma * np.max(Q_reward[new_state, :]) - Q_reward[state, action])
                
    # Our new state is state
    state = new_state
        
    # If done : finish episode
    if done == True:
      # print('solved!') 
      break

  episode += 1
    # Reduce epsilon (because we need less and less exploration)
  epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode)

reward_board = []
action_board = []

test_episodes = 10
test_step = 50

for episode in range(test_episodes):
  state = env.reset()
  step = 0
  done = False
  total_rewards = 0
  action_count = 0
  for step in range(test_step):
    action_count += 1
    action = np.argmax(Q_reward[state,:])
        
    new_state, reward, done, info = env.step(action)
        
    total_rewards += reward
        
    if done == True:
      reward_board.append(total_rewards)
      action_board.append(action_count)
      break
    state = new_state


env.close()
reward_board = np.array(reward_board)
action_board = np.array(action_board)
print("Average total reward: ",np.average(reward_board))
print("Average action number: ",np.average(action_board))

Average total reward:  8.5
Average action number:  12.5
time: 3.18 s
